In [9]:
!pip install psycopg2-binary
!pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]


<h1>환경변수 설정</h1>

In [2]:
from dotenv import load_dotenv
import os
import psycopg2
import pandas as pd


POSTGRE_DB_NAME = os.getenv("POSTGRE_DB_NAME")
POSTGRE_DB_USER = os.getenv("POSTGRE_DB_USER")
POSTGRE_DB_HOST = os.getenv("POSTGRE_DB_HOST")
POSTGRE_DB_PORT = os.getenv("POSTGRE_DB_PORT")
POSTGRE_DB_PASSWORD = os.getenv("POSTGRE_DB_PASSWORD")

# env value check
print(POSTGRE_DB_USER)

gorani


In [4]:
# 기존에 만들어놓은 DB 테스트
conn = psycopg2.connect(
        dbname=POSTGRE_DB_NAME,
        user=POSTGRE_DB_USER,
        host=POSTGRE_DB_HOST,
        port=POSTGRE_DB_PORT,
        password=POSTGRE_DB_PASSWORD
    )

try:
    # 비밀번호 없이 연결
    with conn.cursor() as cursor:        
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM documents;")
        
        result = cursor.fetchall()
        print(f"총 결과 행 수: {len(result)}")

        # 컬럼명 가져오기
        columns = [desc[0] for desc in cursor.description]

        # 데이터프레임 변환
        test_df = pd.DataFrame(result, columns=columns)


except Exception as e:
    print(f"Error: {e}")

finally:
    conn.close()
    print('db 연결 종료!')

test_df

총 결과 행 수: 3
db 연결 종료!


,id,title,content
0,1,첫 번째 문서,고양이와 강아지가 친구가 되었습니다.
1,2,두 번째 문서,강아지는 공기놀이를 좋아합니다.
2,3,세 번째 문서,고양이는 나무 위에서 낮잠을 잤습니다.


<h1>테이블 생성</h1>

<h1></h1>

In [62]:
conn = psycopg2.connect(
        dbname=POSTGRE_DB_NAME,
        user=POSTGRE_DB_USER,
        host=POSTGRE_DB_HOST,
        port=POSTGRE_DB_PORT,
        password=POSTGRE_DB_PASSWORD
    )

create_table_trag0100 = '''
CREATE TABLE TRAGFILE0100 (
    FILE_SEQ        SERIAL      NOT NULL,
    CLSF_CD         INTEGER,
    TYP_CD          INTEGER,
    FILE_NM         TEXT,
    EXT             VARCHAR(10),
    FILE_PATH       TEXT,
    REGIST_ID       VARCHAR(200),
    REGIST_DT       TIMESTAMP,
    MODIFY_ID       VARCHAR(200),
    MODIFY_DT       TIMESTAMP,
    PRIMARY KEY (FILE_SEQ)
);
'''

# SQL for adding comments (optional)
add_comments_sql = '''
COMMENT ON TABLE TRAGFILE0100 IS '첨부파일';
COMMENT ON COLUMN TRAGFILE0100.FILE_SEQ IS '파일순서';
COMMENT ON COLUMN TRAGFILE0100.CLSF_CD IS '구분코드';
COMMENT ON COLUMN TRAGFILE0100.TYP_CD IS '종류코드';
COMMENT ON COLUMN TRAGFILE0100.FILE_NM IS '파일명';
COMMENT ON COLUMN TRAGFILE0100.EXT IS '확장자';
COMMENT ON COLUMN TRAGFILE0100.FILE_PATH IS '파일위치';
COMMENT ON COLUMN TRAGFILE0100.REGIST_ID IS '등록자ID';
COMMENT ON COLUMN TRAGFILE0100.REGIST_DT IS '등록자일시';
COMMENT ON COLUMN TRAGFILE0100.MODIFY_ID IS '수정자ID';
COMMENT ON COLUMN TRAGFILE0100.MODIFY_DT IS '수정자일시';
'''

# MySQL 전용 문법: DEFAULT CHARSET=utf8mb3 COLLATE=utf8mb3_unicode_ci는 MySQL 문법입니다. PostgreSQL에서는 문자 인코딩과 콜레이션을 테이블 단위로 지정하지 않습니다. 이는 데이터베이스 생성 시 설정됩니다.
# COMMENT 문법: MySQL의 COMMENT는 PostgreSQL에서 지원되지 않습니다. 대신 COMMENT ON 명령어를 별도로 사용하거나, 테이블 생성 시 주석을 생략합니다.
# 데이터 타입: VARCHAR(1000)은 PostgreSQL에서 TEXT로 대체 가능합니다. PostgreSQL에서는 VARCHAR 길이 제한이 없어도 TEXT를 사용하는 경우가 많습니다.
# 인덱스 및 제약조건: ALTER TABLE ... ADD CONSTRAINT PRIMARY KEY는 올바르지만, 테이블 생성 시 PRIMARY KEY를 직접 정의하는 것이 더 간결합니다.


try:
    # 비밀번호 없이 연결
    with conn.cursor() as cursor:        
        cursor = conn.cursor()

        cursor.execute(create_table_trag0100)
        print("테이블 생성 완료!")

        cursor.execute(add_comments_sql)
        print("Comment 추가 완료!")

        conn.commit()

except Exception as e:
    print(f"Error: {e}")
    conn.rollback()

finally:
    conn.close()
    print('db 연결 종료!')

테이블 생성 완료!
Comment 추가 완료!
db 연결 종료!


In [63]:
conn = psycopg2.connect(
        dbname=POSTGRE_DB_NAME,
        user=POSTGRE_DB_USER,
        host=POSTGRE_DB_HOST,
        port=POSTGRE_DB_PORT,
        password=POSTGRE_DB_PASSWORD
    )

create_table_trag0102 = '''
CREATE TABLE TRAGFILE0102 (
    PASSAGE_SEQ     SERIAL      NOT NULL,
    FILE_SEQ        INTEGER     NOT NULL,
    CONT            TEXT,
    PAGE_NO_CHST    VARCHAR(200),
    REGIST_ID       VARCHAR(200),
    REGIST_DT       TIMESTAMP,
    MODIFY_ID       VARCHAR(200),
    MODIFY_DT       TIMESTAMP,
    PRIMARY KEY (PASSAGE_SEQ, FILE_SEQ),
    FOREIGN KEY (FILE_SEQ) REFERENCES TRAGFILE0100 (FILE_SEQ)
);
'''

# SQL for adding comments (optional)
add_comments_sql_2 = '''
COMMENT ON TABLE TRAGFILE0102 IS '첨부파일_패시지_결과';
COMMENT ON COLUMN TRAGFILE0102.PASSAGE_SEQ IS '패시지순서';
COMMENT ON COLUMN TRAGFILE0102.FILE_SEQ IS '파일순서';
COMMENT ON COLUMN TRAGFILE0102.CONT IS '내용';
COMMENT ON COLUMN TRAGFILE0102.PAGE_NO_CHST IS '페이지번호';
COMMENT ON COLUMN TRAGFILE0102.REGIST_ID IS '등록자ID';
COMMENT ON COLUMN TRAGFILE0102.REGIST_DT IS '등록자일시';
COMMENT ON COLUMN TRAGFILE0102.MODIFY_ID IS '수정자ID';
COMMENT ON COLUMN TRAGFILE0102.MODIFY_DT IS '수정자일시';
'''



# MySQL 전용 문법: DEFAULT CHARSET=utf8mb3 COLLATE=utf8mb3_unicode_ci는 MySQL 문법입니다. PostgreSQL에서는 문자 인코딩과 콜레이션을 테이블 단위로 지정하지 않습니다. 이는 데이터베이스 생성 시 설정됩니다.
# COMMENT 문법: MySQL의 COMMENT는 PostgreSQL에서 지원되지 않습니다. 대신 COMMENT ON 명령어를 별도로 사용하거나, 테이블 생성 시 주석을 생략합니다.
# 데이터 타입: VARCHAR(1000)은 PostgreSQL에서 TEXT로 대체 가능합니다. PostgreSQL에서는 VARCHAR 길이 제한이 없어도 TEXT를 사용하는 경우가 많습니다.
# 인덱스 및 제약조건: ALTER TABLE ... ADD CONSTRAINT PRIMARY KEY는 올바르지만, 테이블 생성 시 PRIMARY KEY를 직접 정의하는 것이 더 간결합니다.


try:
    # 비밀번호 없이 연결
    with conn.cursor() as cursor:        
        cursor = conn.cursor()

        cursor.execute(create_table_trag0102)
        print("0102 테이블 생성 완료!")

        cursor.execute(add_comments_sql_2)
        print("Comment 추가 완료!")
        
        conn.commit()

except Exception as e:
    print(f"Error: {e}")
    conn.rollback()

finally:
    conn.close()
    print('db 연결 종료!')

0102 테이블 생성 완료!
Comment 추가 완료!
db 연결 종료!


In [64]:
conn = psycopg2.connect(
        dbname=POSTGRE_DB_NAME,
        user=POSTGRE_DB_USER,
        host=POSTGRE_DB_HOST,
        port=POSTGRE_DB_PORT,
        password=POSTGRE_DB_PASSWORD
    )


alter_tsvector_sql = '''
ALTER TABLE TRAGFILE0102 ADD COLUMN cont_tsvector TSVECTOR;
'''

index_tsvector_sql = '''
CREATE INDEX TRAGFILE0102_fts_idx ON TRAGFILE0102 USING GIN (cont_tsvector);
'''

try:
    # 비밀번호 없이 연결
    with conn.cursor() as cursor:        
        cursor = conn.cursor()

        cursor.execute(alter_tsvector_sql)
        print("tsvector 열 추가!")

        cursor.execute(index_tsvector_sql)
        print("Comment 추가 완료!")

        conn.commit()

except Exception as e:
    print(f"Error: {e}")
    conn.rollback()

finally:
    conn.close()
    print('db 연결 종료!')

tsvector 열 추가!
Comment 추가 완료!
db 연결 종료!


<h1>데이터 삽입</h1>
Postgre 에서 Insert할 때, 집어넣는 데이터는 큰따옴표가 아닌 작은따옴표가 들어간다.
차라리 %s 의 바인딩 방식을 사용하는 편이 낫다.

In [65]:
# 3-0. 많이 사용되는 sql 모음
# =============== SELECT ================
select_0100_all_sql = """
select * from TRAGFILE0100;
"""

select_0102_all_sql = """
select * from TRAGFILE0102;
"""

select_0100_0102_join_sql = """
SELECT TF0.FILE_NM, TF0.FILE_SEQ, TF2S.PASSAGE_SEQ, TF2S.CONT, TF2S.PAGE_NO_CHST
FROM TRAGFILE0100 TF0 
INNER JOIN TRAGFILE0102 TF2S
ON TF0.FILE_SEQ = TF2S.FILE_SEQ
ORDER BY TF0.FILE_SEQ ASC;
"""

select_0100_0102_join_tsvector_sql = """
SELECT TF0.FILE_NM, TF0.FILE_SEQ, TF2S.PASSAGE_SEQ, TF2S.CONT, TF2S.PAGE_NO_CHST, TF2S.CONT_TSVECTOR
FROM TRAGFILE0100 TF0 
INNER JOIN TRAGFILE0102 TF2S
ON TF0.FILE_SEQ = TF2S.FILE_SEQ
ORDER BY TF0.FILE_SEQ ASC;
"""

select_find_one_file_sql = """
SELECT FILE_SEQ
FROM TRAGFILE0100
WHERE REPLACE(TRIM(FILE_NM), ' ', '') = REPLACE(TRIM(%s), ' ', '');
"""

select_count_0102_sql = """
select count(*) from TRAGFILE0102;
"""

# =============== INSERT ================

insert_0102_all_sql = """
INSERT INTO TRAGFILE0102
(PASSAGE_SEQ, FILE_SEQ, CONT, PAGE_NO_CHST) VALUES
(%s, %s, %s, %s);
"""

# =============== DELETE ================
delete_0100_all_sql = """
DELETE FROM TRAGFILE0100;
"""

delete_0102_all_sql = """
DELETE FROM TRAGFILE0102;
"""

In [ ]:
df_0100 = pd.read_excel('../data/unit_test/file/커스텀_목록테이블 입력데이터_v0.2_20250203.xlsx', skiprows=2)
# df_0100.head(3)

print(len(df_0100))

clsf_cd = list(df_0100['clsf_cd'])
typ_cd = list(df_0100['typ_cd'])
file_name = list(df_0100['FILE_NM'])
ext = list(df_0100['EXT.1'])

print(len(clsf_cd), len(typ_cd), len(file_name), len(ext))

df_0100.head(3)

In [ ]:
from datetime import datetime

# 3-1.
# 0100에 기본 데이터 insert
conn = psycopg2.connect(
        dbname=POSTGRE_DB_NAME,
        user=POSTGRE_DB_USER,
        host=POSTGRE_DB_HOST,
        port=POSTGRE_DB_PORT,
        password=POSTGRE_DB_PASSWORD
    )

today = datetime.now().strftime("%y%m%d_%H%M%S")

file_check = ''
change_seq = 0

try:
    with conn.cursor() as cursor:
         # 0) DELETE
        cursor.execute(delete_0100_all_sql)

        for idx in range(len(df_0100)):
            try:
                cursor.execute("INSERT INTO TRAGFILE0100 (CLSF_CD, TYP_CD, FILE_NM, EXT) VALUES (%s, %s, %s, %s)",(clsf_cd[idx], typ_cd[idx], file_name[idx], ext[idx]))

            except Exception as e:
                # 중간에 어떤 에러가 나든, 해당 쿼리는 건너뛰고 다음 idx 진행
                print(f"[에러 발생] idx={idx}")
                print("에러 메시지:", e)
                continue

        # 처리 끝난 후 결과 확인
        cursor.execute(select_0100_all_sql)

        result = cursor.fetchall()
        print(f"총 결과 행 수: {len(result)}")

        # 컬럼명 가져오기
        columns = [desc[0] for desc in cursor.description]

        # 데이터프레임 변환
        df_0100_db_result = pd.DataFrame(result, columns=columns)

        # 변경사항 커밋
        conn.commit()

finally:
    conn.close()


df_0100_db_result.to_excel(f'../data/unit_test/result/db_0100_select_result_{today}.xlsx')
df_0100_db_result.tail(3)


In [69]:
# 3-2. delete -> insert -> select(excel export)
# sql 쿼리는 3-0. 에서 가져옴.
conn = psycopg2.connect(
        dbname=POSTGRE_DB_NAME,
        user=POSTGRE_DB_USER,
        host=POSTGRE_DB_HOST,
        port=POSTGRE_DB_PORT,
        password=POSTGRE_DB_PASSWORD
    )

file_check = ''
change_seq = 0
today = datetime.now().strftime("%y%m%d_%H%M%S")

df_0102 = pd.read_excel('../data/unit_test/file/db_insert_passage_250321.xlsx')

print(f'=====데이터를 읽어들였습니다.===== 데이터프레임의 shape는 {df_0102.shape} 입니다.=======')

try:
    with conn.cursor() as cursor:
         # 0) DELETE
        print('====테이블 초기화 작업을 시작합니다.======')
        cursor.execute(delete_0102_all_sql)

        print('====DELETE 완료, original 테이블에서 이제 조회를 시작합니다.=====')

        for idx in range(len(df_0102)):
            try:
                # file check init
                if idx == 0:
                    file_check = df_0102.at[idx, 'Filename']

                file_name = df_0102.at[idx, 'Filename']
                

                # 1) FILE_SEQ 조회
                cursor.execute(select_find_one_file_sql, (file_name,))
                result = cursor.fetchall()  # 여러 행을 가져옴

                if not result:
                    print(f"[에러] 파일명 '{file_name}'에 해당하는 FILE_SEQ 없음! (idx={idx})")
                    # 파일명을 찾지 못하면 건너뛰고 다음으로
                    continue
                
                # psycopg2는 fetchall 반환할 때 이렇게 반환함 ---> [(10,)]
                code = result[0][0]

                # 2) INSERT 처리
                content_value = df_0102.at[idx, 'Content']
                extracted_pages = df_0102.at[idx, 'ExtractedPages']

                # 같은 파일이면 Passage_seq를 이어가고, 다른 파일이면 0으로 리셋
                if file_check == file_name:
                    cursor.execute(insert_0102_all_sql, (change_seq, code, content_value, extracted_pages))
                    conn.commit()
                    change_seq += 1
                else:
                    file_check = file_name
                    change_seq = 0
                    cursor.execute(insert_0102_all_sql, (change_seq, code, content_value, extracted_pages))
                    conn.commit()
                    change_seq += 1

                print(f'[{idx + 1} 행] : {change_seq}-{code}')

            except Exception as e:
                # 중간에 어떤 에러가 나든, 해당 쿼리는 건너뛰고 다음 idx 진행
                print(f"[에러 발생] idx={idx}, filename={file_name}")
                print("에러 메시지:", e)
                continue

        # 처리 끝난 후 결과 확인
        cursor.execute(select_0100_0102_join_sql)

        result = cursor.fetchall()

        # 컬럼명 가져오기
        columns = [desc[0] for desc in cursor.description]

        # 데이터프레임 변환
        df_db_insert_result = pd.DataFrame(result, columns=columns)

        # 변경사항 커밋
        conn.commit()

finally:
    conn.close()

print(f"첫번째 조회 시 저장된 row 수는=====> {len(df_0102)} 입니다!!")
print(f"최종 조회 시 저장된 row 수는=====>: {len(df_db_insert_result)} 입니다!!")
df_db_insert_result.to_excel(f'../data/unit_test/result/db_0102_select_result_{today}.xlsx')
# df_db_insert_result.tail(3)


=====데이터를 읽어들였습니다.===== 데이터프레임의 shape는 (6066, 4) 입니다.=======
====테이블 초기화 작업을 시작합니다.======
====DELETE 완료, original 테이블에서 이제 조회를 시작합니다.=====
[1 행] : 1-3
[2 행] : 2-3
[3 행] : 3-3
[4 행] : 4-3
[5 행] : 5-3
[6 행] : 6-3
[7 행] : 7-3
[8 행] : 8-3
[9 행] : 9-3
[10 행] : 10-3
[11 행] : 11-3
[12 행] : 12-3
[13 행] : 13-3
[14 행] : 14-3
[15 행] : 15-3
[16 행] : 16-3
[17 행] : 17-3
[18 행] : 18-3
[19 행] : 19-3
[20 행] : 20-3
[21 행] : 21-3
[22 행] : 22-3
[23 행] : 23-3
[24 행] : 24-3
[25 행] : 25-3
[26 행] : 26-3
[27 행] : 27-3
[28 행] : 28-3
[29 행] : 29-3
[30 행] : 30-3
[31 행] : 31-3
[32 행] : 32-3
[33 행] : 33-3
[34 행] : 34-3
[35 행] : 35-3
[36 행] : 36-3
[37 행] : 37-3
[38 행] : 38-3
[39 행] : 39-3
[40 행] : 40-3
[41 행] : 41-3
[42 행] : 42-3
[43 행] : 43-3
[44 행] : 44-3
[45 행] : 45-3
[46 행] : 46-3
[47 행] : 47-3
[48 행] : 48-3
[49 행] : 49-3
[50 행] : 50-3
[51 행] : 51-3
[52 행] : 52-3
[53 행] : 53-3
[54 행] : 54-3
[55 행] : 55-3
[56 행] : 56-3
[57 행] : 57-3
[58 행] : 58-3
[59 행] : 59-3
[60 행] : 60-3
[61 행] : 61-3
[62 행] : 62-3
[63 행] : 63

<h1>키워드 변환 작업</h1>

In [ ]:
conn = psycopg2.connect(
        dbname=POSTGRE_DB_NAME,
        user=POSTGRE_DB_USER,
        host=POSTGRE_DB_HOST,
        port=POSTGRE_DB_PORT,
        password=POSTGRE_DB_PASSWORD
    )


update_tsvector_sql = '''
UPDATE TRAGFILE0102 SET cont_tsvector = to_tsvector('korean', cont);
'''

try:
    # 비밀번호 없이 연결
    with conn.cursor() as cursor:        
        cursor = conn.cursor()

        cursor.execute(update_tsvector_sql)
        print('tsvector 업데이트 완료!')

        cursor.execute(select_0100_0102_join_tsvector_sql)

        result = cursor.fetchall()

        # 컬럼명 가져오기
        columns = [desc[0] for desc in cursor.description]

        # 데이터프레임 변환
        df_db_tsvector_result = pd.DataFrame(result, columns=columns)

        # 변경사항 커밋
        conn.commit()

except Exception as e:
    print(f"Error: {e}")
    conn.rollback()

finally:
    conn.close()
    print('db 연결 종료!')


df_db_tsvector_result.to_excel(f'../data/unit_test/result/db_0102_tsvector_result_{today}.xlsx')
df_db_tsvector_result.tail(3)

In [73]:
tsvector = df_db_tsvector_result['cont_tsvector'][0]
tsvector

"'1':34,89,125,129 '14':180,250,266 '2':1,59,110 '23':251 '24':86 '3':103,151,228,279,290 '4':24,43,90,126,164,242 '500':118,305 '6':7 '7':45,92 '8':26,128,166 '각':112 '감독관':147,275 '개':152 '것':54,221 '결과':237 '경우':184 '고령자':19,81,160 '고용':14,20,35,69,76,82,132,191,203,284 '과태료':88,107,122,185,272,294,309 '관련':12 '관서':196,208 '관하':22,84 '구제':5 '규정':149,277 '근로':146,274 '근로관계':154 '금지':18,80 '기간':179,265 '기준':159,177 '노동':195,207 '노동부':36,70,133,192,204,285 '노동청':256 '다음':111 '대장':249 '대하':214,298 '등':33 '따르':47,94,131,143 '만':119,306 '명령':3,9,28,49,67,96,170,189,211,216,231,260,288,297 '받':212 '법':155 '법률':13,23,85 '법조문':163 '별':153 '별지':227,241 '별표':150,278 '보':258 '보고서':234 '본부':257 '부과':108,123,273,295,310 '부록':226,240 '불응':175,270,303 '사건':247 '사례':168 '사업주':42,65,187,213 '사유':99,141 '사항':157 '상':15,77 '상황':30,51,136,172,218,236,262,300 '서식':229,243 '수':57,74,201,224 '시정':2,8,27,48,66,95,169,178,183,188,210,215,230,246,259,287,296 '신고':72,199 '씨':271,304 '어느':114 '없이':100,142,283 '

<h1>검색 테스트 1</h1>
일단, 조건을 직접 하드코딩해서 넣어본다.

In [81]:
# 특정 키워드로 입력하여 필터링 검색
conn = psycopg2.connect(
        dbname=POSTGRE_DB_NAME,
        user=POSTGRE_DB_USER,
        host=POSTGRE_DB_HOST,
        port=POSTGRE_DB_PORT,
        password=POSTGRE_DB_PASSWORD
    )


search_tsvector_sql = '''
SELECT cont_tsvector
FROM tragfile0102
WHERE cont_tsvector @@ to_tsquery('korean', '동물');
'''

try:
    # 비밀번호 없이 연결
    with conn.cursor() as cursor:        
        cursor = conn.cursor()

        cursor.execute(search_tsvector_sql)
        print('tsvector 업데이트 완료!')

        result = cursor.fetchall()

        # 컬럼명 가져오기
        columns = [desc[0] for desc in cursor.description]

        # 데이터프레임 변환
        tsvector_search_result = pd.DataFrame(result, columns=columns)

        # 변경사항 커밋
        conn.commit()

except Exception as e:
    print(f"Error: {e}")
    conn.rollback()

finally:
    conn.close()
    print('db 연결 종료!')

print('검색 건수!!! ===> ', len(tsvector_search_result))
tsvector_search_result.tail(3)

tsvector 업데이트 완료!
db 연결 종료!
검색 건수!!! ===>  27


,cont_tsvector
24,"'1':16,58,120,142,144,160,168,176,211 '11':20 ..."
25,"'1':37,47,49,66,86,106,164,177,268,272 '2':45,..."
26,"'-13':546 '01':27,163 '012':252 '0121':348 '01..."


In [ ]:
# '동물' 이라는 단어가 들어가 있다.
list(tsvector_search_result['cont_tsvector'])[0]

"'1':8,11,19,44,54,56,73,93,113,171,185,210,217,226,241,268,324,340,344,350,353,454,456,467,514,525,550,592,640,747,751,759 '10':273,597,720 '12':351,757 '14':728 '15':283,609,737 '16':310,367,660,680,776 '2':52,63,71,184,219,234,243,275,288,295,312,320,366,395,452,516,552,599,621,636,662,710,739,804,826 '21':297,646 '3':83,91,200,204,250,259,299,315,494,498,561,648,665,736 '34':286,619 '4':103,111,142,221,245,282,518,554 '42':16,389,465,798,820 '43':190,293,391,393,800,802,822,824 '44':330,397,806,828 '45':372,815 '5':1,34,38,123,131,156,211,213,235,237,436,440,480,484,504,506,540,542 '6':139,215,239,512,548 '7':153,173,224,248,266,523,559,590 '8':168 '9':711 '각':21,194,334,469,693 '감독':430,898 '개관':10 '개인':378,386,410,418,845,854,862,870,878,886 '갱신':278,602 '거짓':260,584 '것':700 '게을리':431,899 '경우':434,490,631,833,876,881,902 '계속':363,770 '고기':161 '과':414,415,866,867 '관련':670 '관하':50,67,87,107,127,308,388,426,450,658,678,856,894 '귀책':625 '규정':175,713,722,730,741,817 '그':169,261,381,38

<h1>검색 테스트 2</h1>
질문을 공백 기준으로 쪼개서 넣어본다. 필자 생각으로는 뭔가 심심하다.

In [ ]:
# 사용자의 쿼리가 들어갈 때
# 필자 생각으로는 좀 그렇다.. 차라리 질문을 형태소 분석 시켜서 의미 있는 단어를 and 시키면 어떨까 생각을 해봄
user_query = "동물 보호"
query_condition = ' & '.join(user_query.split())

# 특정 키워드로 입력하여 필터링 검색
conn = psycopg2.connect(
        dbname=POSTGRE_DB_NAME,
        user=POSTGRE_DB_USER,
        host=POSTGRE_DB_HOST,
        port=POSTGRE_DB_PORT,
        password=POSTGRE_DB_PASSWORD
    )


search_tsvector_sql = '''
SELECT cont_tsvector
FROM tragfile0102
WHERE cont_tsvector @@ to_tsquery('korean', %s);
'''

try:
    # 비밀번호 없이 연결
    with conn.cursor() as cursor:        
        cursor = conn.cursor()

        # 두번째 인자를 튜플로 받아야 함.
        cursor.execute(search_tsvector_sql, (query_condition,))
        print('tsvector 업데이트 완료!')

        result = cursor.fetchall()

        # 컬럼명 가져오기
        columns = [desc[0] for desc in cursor.description]

        # 데이터프레임 변환
        tsvector_search_result = pd.DataFrame(result, columns=columns)

        # 변경사항 커밋
        conn.commit()

except Exception as e:
    print(f"Error: {e}")
    conn.rollback()

finally:
    conn.close()
    print('db 연결 종료!')

print('검색 건수!!! ===> ', len(tsvector_search_result))
tsvector_search_result.tail(3)

tsvector 업데이트 완료!
db 연결 종료!
검색 건수!!! ===>  9


,cont_tsvector
6,"'02':17 '1':30 '2':2,111,181 '3':88 'emerg':26..."
7,"'1':16,58,120,142,144,160,168,176,211 '11':20 ..."
8,"'-13':546 '01':27,163 '012':252 '0121':348 '01..."


In [79]:
print(tsvector_search_result['cont_tsvector'][0])

'1':8,11,19,44,54,56,73,93,113,171,185,210,217,226,241,268,324,340,344,350,353,454,456,467,514,525,550,592,640,747,751,759 '10':273,597,720 '12':351,757 '14':728 '15':283,609,737 '16':310,367,660,680,776 '2':52,63,71,184,219,234,243,275,288,295,312,320,366,395,452,516,552,599,621,636,662,710,739,804,826 '21':297,646 '3':83,91,200,204,250,259,299,315,494,498,561,648,665,736 '34':286,619 '4':103,111,142,221,245,282,518,554 '42':16,389,465,798,820 '43':190,293,391,393,800,802,822,824 '44':330,397,806,828 '45':372,815 '5':1,34,38,123,131,156,211,213,235,237,436,440,480,484,504,506,540,542 '6':139,215,239,512,548 '7':153,173,224,248,266,523,559,590 '8':168 '9':711 '각':21,194,334,469,693 '감독':430,898 '개관':10 '개인':378,386,410,418,845,854,862,870,878,886 '갱신':278,602 '거짓':260,584 '것':700 '게을리':431,899 '경우':434,490,631,833,876,881,902 '계속':363,770 '고기':161 '과':414,415,866,867 '관련':670 '관하':50,67,87,107,127,308,388,426,450,658,678,856,894 '귀책':625 '규정':175,713,722,730,741,817 '그':169,261,381,384

<h1>검색 테스트 3</h1>
질문을 tsvector로 명사를 추출한다.

추출한 명사를 정제한 후, join 시켜 조건으로 추가한다.

한 음절 검색은 영향도가 있을 수 있으므로, 두 음절부터 가능하도록 조건을 걸어놓는다.
숫자의 경우도 정규식으로 필터링해도 될 것 같다.

In [29]:
# 변형 방안 :: 질문을 먼저 to_tsvector 에 넣어서 단어로 만든다.
# 단어를 정제하여 길이가 2이상인 것만 뽑아낸다.
# 그 결과를 검색 조건으로 and 시켜 넣는다.
user_query = "동물 보호 조치"

# 특정 키워드로 입력하여 필터링 검색
conn = psycopg2.connect(
        dbname=POSTGRE_DB_NAME,
        user=POSTGRE_DB_USER,
        host=POSTGRE_DB_HOST,
        port=POSTGRE_DB_PORT,
        password=POSTGRE_DB_PASSWORD
    )


search_tsvector_sql = '''
SELECT to_tsvector('korean', %s);
'''

try:
    # 비밀번호 없이 연결
    with conn.cursor() as cursor:        
        cursor = conn.cursor()

        # 두번째 인자를 튜플로 받아야 함.
        cursor.execute(search_tsvector_sql, (user_query,))
        print('tsvector 업데이트 완료!')

        result = cursor.fetchall()
        print(result)

        # 컬럼명 가져오기
        columns = [desc[0] for desc in cursor.description]

        # 데이터프레임 변환
        tsvector_search_result = pd.DataFrame(result, columns=columns)

        # 변경사항 커밋
        conn.commit()

except Exception as e:
    print(f"Error: {e}")
    conn.rollback()

finally:
    conn.close()
    print('db 연결 종료!')

print('검색 건수!!! ===> ', len(tsvector_search_result))
tsvector_search_result.tail(3)

tsvector 업데이트 완료!
[("'동물':1 '보호':2 '조치':3",)]
db 연결 종료!
검색 건수!!! ===>  1


,to_tsvector
0,'동물':1 '보호':2 '조치':3


In [31]:
# 단어만 추출 (2글자 이상만?) 그리고 다시 검색을 날리면 되지 않을까?
d = result[0][0].split(":")

print([c.split() for c in d])
print([c.split()[-1] for c in d][:-1])
print([k.replace("'","") for k in [c.split()[-1] for c in d][:-1]])


# 이 값을 최종적으로 쿼리에 날려야 할 듯.
final_data = [u for u in [k.replace("'","") for k in [c.split()[-1] for c in d][:-1]] if len(u) >= 2]
final_data


[["'동물'"], ['1', "'보호'"], ['2', "'조치'"], ['3']]
["'동물'", "'보호'", "'조치'"]
['동물', '보호', '조치']


['동물', '보호', '조치']

In [26]:
### 앞에서 받는 final_data를 join 시켜 넣는다.

# 특정 키워드로 입력하여 필터링 검색
conn = psycopg2.connect(
        dbname=POSTGRE_DB_NAME,
        user=POSTGRE_DB_USER,
        host=POSTGRE_DB_HOST,
        port=POSTGRE_DB_PORT,
        password=POSTGRE_DB_PASSWORD
    )

final_join_data = '&'.join(final_data)

search_tsvector_sql = '''
SELECT cont_tsvector
FROM tragfile0102
WHERE cont_tsvector @@ to_tsquery('korean', %s);
'''

try:
    # 비밀번호 없이 연결
    with conn.cursor() as cursor:        
        cursor = conn.cursor()

        # 두번째 인자를 튜플로 받아야 함.
        cursor.execute(search_tsvector_sql, (final_join_data,))

        result = cursor.fetchall()

        # 컬럼명 가져오기
        columns = [desc[0] for desc in cursor.description]

        # 데이터프레임 변환
        tsvector_search_result = pd.DataFrame(result, columns=columns)

        # 변경사항 커밋
        conn.commit()

except Exception as e:
    print(f"Error: {e}")
    conn.rollback()

finally:
    conn.close()
    print('db 연결 종료!')

print('검색 건수!!! ===> ', len(tsvector_search_result))
tsvector_search_result.tail(3)

db 연결 종료!
검색 건수!!! ===>  4


,cont_tsvector
1,"'1':48,130,154,166,171,178,195,202,222,229,266..."
2,"'02':17 '1':30 '2':2,111,181 '3':88 'emerg':26..."
3,"'1':16,58,120,142,144,160,168,176,211 '11':20 ..."


In [27]:
tsvector_search_result['cont_tsvector'][0]

"'1':8,11,19,44,54,56,73,93,113,171,185,210,217,226,241,268,324,340,344,350,353,454,456,467,514,525,550,592,640,747,751,759 '10':273,597,720 '12':351,757 '14':728 '15':283,609,737 '16':310,367,660,680,776 '2':52,63,71,184,219,234,243,275,288,295,312,320,366,395,452,516,552,599,621,636,662,710,739,804,826 '21':297,646 '3':83,91,200,204,250,259,299,315,494,498,561,648,665,736 '34':286,619 '4':103,111,142,221,245,282,518,554 '42':16,389,465,798,820 '43':190,293,391,393,800,802,822,824 '44':330,397,806,828 '45':372,815 '5':1,34,38,123,131,156,211,213,235,237,436,440,480,484,504,506,540,542 '6':139,215,239,512,548 '7':153,173,224,248,266,523,559,590 '8':168 '9':711 '각':21,194,334,469,693 '감독':430,898 '개관':10 '개인':378,386,410,418,845,854,862,870,878,886 '갱신':278,602 '거짓':260,584 '것':700 '게을리':431,899 '경우':434,490,631,833,876,881,902 '계속':363,770 '고기':161 '과':414,415,866,867 '관련':670 '관하':50,67,87,107,127,308,388,426,450,658,678,856,894 '귀책':625 '규정':175,713,722,730,741,817 '그':169,261,381,38

<h1>Full Code</h1>

In [74]:
from dotenv import load_dotenv
import os
import psycopg2
import pandas as pd

load_dotenv()

POSTGRE_DB_NAME = os.getenv("POSTGRE_DB_NAME")
POSTGRE_DB_USER = os.getenv("POSTGRE_DB_USER")
POSTGRE_DB_HOST = os.getenv("POSTGRE_DB_HOST")
POSTGRE_DB_PORT = os.getenv("POSTGRE_DB_PORT")
POSTGRE_DB_PASSWORD = os.getenv("POSTGRE_DB_PASSWORD")

def postgre_db_connect():
    return psycopg2.connect(
        dbname=POSTGRE_DB_NAME,
        user=POSTGRE_DB_USER,
        host=POSTGRE_DB_HOST,
        port=POSTGRE_DB_PORT,
        password=POSTGRE_DB_PASSWORD
    )

def query_execute(conn, sql, data_tuple):
    try:
        with conn.cursor() as cursor:
            cursor.execute(sql, data_tuple)
            result = cursor.fetchall()
            columns = [desc[0] for desc in cursor.description]
        return result, columns
    except Exception as e:
        print(f"Error: {e}")
        conn.rollback()
        return None, None

def keyword_refinement(keyword_info):
    keyword_list = keyword_info.split(':')
    keyword_list = [keyword.split()[-1] for keyword in keyword_list][:-1]
    keyword_list = [keyword.replace("'", "") for keyword in keyword_list]
    return '&'.join([keyword for keyword in keyword_list if len(keyword) >= 2])

def keyword_search(user_query):
    get_keyword_sql = "SELECT to_tsvector('korean', %s);"
    # detail query
    # search_tsvector_sql = '''
    # SELECT TF0.FILE_NM, TF2.cont, TF2.cont_tsvector, ts_rank(cont_tsvector, to_tsquery('korean', %s)) AS rank
    # FROM TRAGFILE0102 TF2
    # INNER JOIN TRAGFILE0100 TF0
    # ON TF0.FILE_SEQ = TF2.FILE_SEQ
    # WHERE cont_tsvector @@ to_tsquery('korean', %s);
    # '''

    # simple query
    search_tsvector_sql = '''
    SELECT cont_tsvector, ts_rank(cont_tsvector, to_tsquery('korean', %s)) AS rank
    FROM TRAGFILE0102
    WHERE cont_tsvector @@ to_tsquery('korean', %s);
    '''

    conn = postgre_db_connect()
    try:
        keyword_info, _ = query_execute(conn, get_keyword_sql, (user_query,))
        if not keyword_info:
            print("No keyword info found.")
            return None

        keyword_info = keyword_info[0][0]
        keyword_condition = keyword_refinement(keyword_info)
        print('==== condition ==== :: ', keyword_condition)

        result, columns = query_execute(conn, search_tsvector_sql, (keyword_condition, keyword_condition,))
        if result is None:
            print("No search result found.")
            return None

        return pd.DataFrame(result, columns=columns)
    finally:
        conn.close()
        print('DB 연결 종료!')

# 실행 예시
df = keyword_search("동물병원")
df

==== condition ==== ::  동물&병원
DB 연결 종료!


,cont_tsvector,rank
0,'-1145':488 '-1596':220 '-1764':589 '-1947':74...,2.220446e-15
1,"'1':26,28,69 '3':24,246 '34':231 '4':7,228 '59...",4.418688e-09


In [75]:
df['cont_tsvector'][1]

"'1':26,28,69 '3':24,246 '34':231 '4':7,228 '59':67,207,226,244,259 '600':177 '63':173 'ii':38,121 'iii':41,135 'iv':44,150 'v':49 '가하':88 '감독':160,235 '감시':136,290 '개간':111 '개개인':211 '것':79,89,100,223,255 '경':187 '경우':59,165,257,271 '경작':110 '고용':144,180,304 '관계':221 '관계없이':158 '관련':47,213 '관리':159,195,234 '관하':168,300 '구급차':191 '규정':169,209,215,301,316 '그':117,129,256 '근로':96,139,204 '근로자':54,155,176,190,198,210,265,287 '기밀':162,236 '기준':205 '기타':45 '길이':76 '내용':220 '노동부':145,305 '노선':31,32 '농업':119 '단속':137,291 '대상자':319 '대통령령':151 '대표':55,266 '동물':122 '동식물':125 '들':288 '등':192 '따르':30 '때문':313 '말':90 '면':268 '명':179 '바':311 '밖':118,130 '받':148,310 '배제':303 '법':66,172,206,225,243,258 '법률':77 '법상':166 '법제':23 '변경':3,10,62,73,103,275 '병원':181,185 '보건':50 '본문':261 '부여':201 '사건':280 '사안':202 '사업':22,36,116,120,128,134,156,219,250 '사용':52,142,183,216,263 '사육':123 '상시':175 '서':267 '서면':56,283 '서비스업':48 '수':64,105,241,253,277,296,322 '수산':124,133 '수상':39 '수행':189,197 '승인':147,307 '시간':2,9,